## MNIST (3) Subclassing 활용

마지막으로 Subclassing 방법입니다.   
keras.Model을 상속받은 클래스를 만드는 것입니다.  
__init__() 메서드 안에서 레이어를 선언하고, call() 메서드 안에서 forward propagation을 구현하는 방식임을 기억해 주세요.  
Functional 방식과 비교하자면, call()의 입력이 Input이고, call()의 리턴값이 Output이 되는 것입니다.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from datetime import datetime

In [2]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = tf.expand_dims(x_train, axis = 3)
x_test = tf.expand_dims(x_test, axis = 3)

print(len(x_train), len(x_test))

Metal device set to: Apple M1 Pro
60000 10000


In [5]:
data_size = x_train.shape[1:]

In [6]:
data_size

TensorShape([28, 28, 1])

In [13]:
# Subclassing을 활용한 Model을 구성해주세요.


# 여기에 모델을 구성해주세요

class CustomModel(keras.Model):
    ##[[YOUR CODE]]
    """
    Spec:
    0. keras.Model 을 상속받았으며, __init__()와 call() 메서드를 가진 모델 클래스
    1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
    2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
    3. Flatten 레이어
    4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
    5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
    6. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output
    """
        
    def __init__(self):        
        super(CustomModel,self).__init__()

        self.conv1 = keras.layers.Conv2D(32,(3,3),activation='relu')
        self.conv2 = keras.layers.Conv2D(64,(3,3),activation='relu')
        self.flatten = keras.layers.Flatten()
        self.dense1 = keras.layers.Dense(128,activation='relu')
        self.dense2 = keras.layers.Dense(10,activation='softmax')

        
    def call(self,inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        outputs = self.dense2(x)
        return outputs
        
    def summary(self):
        x = keras.Input(shape = (28,28,1))
        model = keras.Model(inputs=[x], outputs = self.call(x))
        return model.summary()


model = CustomModel()

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 conv2d_13 (Conv2D)          (None, 24, 24, 64)        18496     
                                                                 
 flatten_6 (Flatten)         (None, 36864)             0         
                                                                 
 dense_12 (Dense)            (None, 128)               4718720   
                                                                 
 dense_13 (Dense)            (None, 10)                1290      
                                                                 
Total params: 4,738,826
Trainable params: 4,738,826
Non-train

In [14]:
from datetime import datetime
start_time = datetime.now()
print(" Started on: ", start_time.strftime("%I:%M:%S %p"))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2)

model.evaluate(x_test,  y_test, verbose=2)



end_time = datetime.now()
print("Job Completed on: ", end_time.strftime("%I:%M:%S %p"))

execution_time = end_time - start_time
print("Job Execution Time: ", execution_time)


 Started on:  02:43:18 PM
Epoch 1/2


2023-05-11 14:43:19.092955: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 [==============================] - 22s 11ms/step - loss: 0.1042 - accuracy: 0.9674
Epoch 2/2
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0331 - accuracy: 0.9893
313/313 - 2s - loss: 0.0422 - accuracy: 0.9859 - 2s/epoch - 7ms/step
Job Completed on:  02:44:04 PM
Job Execution Time:  0:00:45.310898


In [15]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 2s - loss: 0.0422 - accuracy: 0.9859 - 2s/epoch - 7ms/step


[0.04217268154025078, 0.9859000444412231]

3